In [1]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx, to_scipy_sparse_matrix
import networkx as nx
import numpy as np
from scipy.linalg import pinv
from math import inf
from numba import jit, int64

def choose_edge_to_add(x, edge_index, degrees):
    # chooses edge (u, v) to add which minimizes y[u]*y[v]
    n = x.size
    m = edge_index.shape[1]
    y = x / ((degrees + 1) ** 0.5)
    products = np.outer(y, y)
    for i in range(m):
        u = edge_index[0, i]
        v = edge_index[1, i]
        products[u, v] = inf
    for i in range(n):
        products[i, i] = inf
    smallest_product = np.argmin(products)
    return (smallest_product % n, smallest_product // n)

def compute_degrees(edge_index, num_nodes=None):
    # returns array of degrees of all nodes
    if num_nodes is None:
        num_nodes = np.max(edge_index) + 1
    degrees = np.zeros(num_nodes)
    m = edge_index.shape[1]
    for i in range(m):
        degrees[edge_index[0, i]] += 1
    return degrees

def add_edge(edge_index, u, v):
    new_edge = np.array([[u, v],[v, u]])
    return np.concatenate((edge_index, new_edge), axis=1)

def adj_matrix_multiply(edge_index, x):
    # given an edge_index, computes Ax, where A is the corresponding adjacency matrix
    n = x.size
    y = np.zeros(n)
    m = edge_index.shape[1]
    for i in range(m):
        u = edge_index[0, i]
        v = edge_index[1, i]
        y[u] += x[v]
    return y

def compute_spectral_gap(edge_index, x):
	m = edge_index.shape[1]
	n = np.max(edge_index) + 1
	degrees = compute_degrees(edge_index, num_nodes=n)
	y = adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
	for i in range(n):
		if x[i] > 1e-9:
			return 1 - y[i]/x[i]
	return 0.

def _edge_rewire(edge_index, edge_type, x=None, num_iterations=50, initial_power_iters=50):
	m = edge_index.shape[1]
	n = np.max(edge_index) + 1
	if x is None:
		x = 2 * np.random.random(n) - 1
	degrees = compute_degrees(edge_index, num_nodes=n)
	for i in range(initial_power_iters):
		x = x - x.dot(degrees ** 0.5) * (degrees ** 0.5)/sum(degrees)
		y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
		x = y / np.linalg.norm(y)
	for I in range(num_iterations):
		i, j = choose_edge_to_add(x, edge_index, degrees=degrees)
		edge_index = add_edge(edge_index, i, j)
		degrees[i] += 1
		degrees[j] += 1
		edge_type = np.append(edge_type, 1)
		edge_type = np.append(edge_type, 1)
		x = x - x.dot(degrees ** 0.5) * (degrees ** 0.5)/sum(degrees)
		y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
		x = y / np.linalg.norm(y)
	return edge_index, edge_type, x

def edge_rewire(edge_index, x=None, edge_type=None, num_iterations=50, initial_power_iters=5):
    m = edge_index.shape[1]
    n = np.max(edge_index) + 1
    if x is None:
        x = 2 * np.random.random(n) - 1
    if edge_type is None:
        edge_type = np.zeros(m, dtype=np.int64)
    return _edge_rewire(edge_index, edge_type=edge_type, x=x, num_iterations=num_iterations, initial_power_iters=initial_power_iters)


In [2]:

# Apply the FOSR method
def apply_fosr(edge_index, rewire_fraction=0.1, min_iterations=10, max_iterations=1000):
    edge_type = np.zeros(edge_index.shape[1], dtype=np.int64)
    n = np.max(edge_index) + 1
    x = 2 * np.random.random(n) - 1
    
    # Calculate num_iterations based on the number of edges and rewire_fraction
    num_edges = edge_index.shape[1]
    num_iterations = int(num_edges * rewire_fraction)
    
    # Ensure num_iterations is within a reasonable range
    num_iterations = max(min_iterations, min(num_iterations, max_iterations))
    
    print(f"Number of iterations for FOSR: {num_iterations}")
    
    new_edge_index, new_edge_type, _ = edge_rewire(
        edge_index, 
        x=x, 
        edge_type=edge_type, 
        num_iterations=num_iterations, 
        initial_power_iters=5
    )
    
    return new_edge_index, new_edge_type

# Function to apply FOSR to any dataset
def apply_fosr_to_dataset(dataset, rewire_fraction=0.1, min_iterations=10, max_iterations=1000):
    data = dataset[0]
    G = to_networkx(data, to_undirected=True)
    edge_index = np.array(list(G.edges())).T
    
    new_edge_index, new_edge_type = apply_fosr(edge_index, rewire_fraction, min_iterations, max_iterations)
    
    new_G = nx.Graph()
    new_G.add_nodes_from(range(data.num_nodes))
    new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
    new_G.add_edges_from(new_edges)
    
    
    return new_edge_index, new_edge_type

#new_edge_index, new_edge_type = apply_fosr_to_dataset(dataset)
'''
# Create a new graph with the rewired edges
new_G = nx.Graph()
new_G.add_nodes_from(range(data.num_nodes))
new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
new_G.add_edges_from(new_edges)

# Print some statistics
print(f"Original number of edges: {G.number_of_edges()}")
print(f"New number of edges: {new_G.number_of_edges()}")
print(f"Number of added edges: {new_G.number_of_edges() - G.number_of_edges()}")'''



'\n# Create a new graph with the rewired edges\nnew_G = nx.Graph()\nnew_G.add_nodes_from(range(data.num_nodes))\nnew_edges = list(zip(new_edge_index[0], new_edge_index[1]))\nnew_G.add_edges_from(new_edges)\n\n# Print some statistics\nprint(f"Original number of edges: {G.number_of_edges()}")\nprint(f"New number of edges: {new_G.number_of_edges()}")\nprint(f"Number of added edges: {new_G.number_of_edges() - G.number_of_edges()}")'

In [3]:
def compute_commute_time(graph):
    """
    Compute the commute time for each pair of nodes in a graph.
    :param graph: A NetworkX graph object.
    :return: Commute time matrix (numpy array).
    """
    # Convert graph to adjacency matrix (scipy sparse format)
    adj_matrix = nx.adjacency_matrix(graph)
    
    # Compute degree matrix
    degrees = np.array(adj_matrix.sum(axis=1)).flatten()
    D = np.diag(degrees)
    
    # Compute Laplacian matrix L = D - A
    L = D - adj_matrix.toarray()
    
    # Compute the pseudoinverse of the Laplacian
    L_pseudo = pinv(L)
    
    # Compute commute time between all pairs of nodes
    num_nodes = L.shape[0]
    commute_time = np.zeros((num_nodes, num_nodes))
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            commute_time[i, j] = L_pseudo[i, i] + L_pseudo[j, j] - 2 * L_pseudo[i, j]
    
    return commute_time


def aggregate_commute_times(graph):
    """
    Aggregate the commute times for a single graph.
    :param graph: A NetworkX graph object.
    :return: Average commute time across the graph.
    """
    commute_times = compute_commute_time(graph)
    
    # Get upper triangular part of the commute time matrix (since it's symmetric)
    upper_triangle_indices = np.triu_indices_from(commute_times, k=1)
    commute_times_upper = commute_times[upper_triangle_indices]
    
    # Return average commute time
    return np.mean(commute_times_upper)

In [4]:
# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Convert to NetworkX graph
G = to_networkx(data, to_undirected=True)

# Get edge index from the graph
edge_index = np.array(list(G.edges())).T

# Create a new graph with the rewired edges
new_G = nx.Graph()
new_G.add_nodes_from(range(data.num_nodes))
new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
new_G.add_edges_from(new_edges)

# Print some statistics
print(f"Original number of edges: {G.number_of_edges()}")
print(f"New number of edges: {new_G.number_of_edges()}")
print(f"Number of added edges: {new_G.number_of_edges() - G.number_of_edges()}")



new_edge_index, new_edge_type = apply_fosr_to_dataset(dataset)

# Create a new graph with the rewired edges
G_fosr = nx.Graph()
G_fosr.add_nodes_from(range(data.num_nodes))
new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
G_fosr.add_edges_from(new_edges)

# Calculate aggregate commute times for both graphs
agg_commute_time_original = aggregate_commute_times(G_original)
agg_commute_time_fosr = aggregate_commute_times(G_fosr)

print(f"Original Graph - Aggregate Commute Time: {agg_commute_time_original:.4f}")
print(f"FOSR-modified Graph - Aggregate Commute Time: {agg_commute_time_fosr:.4f}")

# Calculate the percentage change in aggregate commute time
percent_change = ((agg_commute_time_fosr - agg_commute_time_original) / agg_commute_time_original) * 100

print(f"Percentage change in Aggregate Commute Time: {percent_change:.2f}%")

NameError: name 'new_edge_index' is not defined

In [5]:
# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Convert to NetworkX graph
G = to_networkx(data, to_undirected=True)

# Get edge index from the graph
edge_index = np.array(list(G.edges())).T

# Apply FOSR to get new edge index
new_edge_index, new_edge_type = apply_fosr_to_dataset(dataset)

# Create a new graph with the rewired edges
new_G = nx.Graph()
new_G.add_nodes_from(range(data.num_nodes))
new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
new_G.add_edges_from(new_edges)

# Print some statistics
print(f"Original number of edges: {G.number_of_edges()}")
print(f"New number of edges: {new_G.number_of_edges()}")
print(f"Number of added edges: {new_G.number_of_edges() - G.number_of_edges()}")

# Calculate aggregate commute times for both graphs
G_original = G  # Rename G to G_original for clarity
G_fosr = new_G  # Rename new_G to G_fosr for consistency

agg_commute_time_original = aggregate_commute_times(G_original)
agg_commute_time_fosr = aggregate_commute_times(G_fosr)

print(f"Original Graph - Aggregate Commute Time: {agg_commute_time_original:.4f}")
print(f"FOSR-modified Graph - Aggregate Commute Time: {agg_commute_time_fosr:.4f}")

# Calculate the percentage change in aggregate commute time
percent_change = ((agg_commute_time_fosr - agg_commute_time_original) / agg_commute_time_original) * 100

print(f"Percentage change in Aggregate Commute Time: {percent_change:.2f}%")

Number of iterations for FOSR: 527


/var/folders/fp/zkb1xvsx6s1gy5dwtqm90krc0000gn/T/ipykernel_6049/4045680862.py:68: RuntimeWarning: divide by zero encountered in divide
  y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
/var/folders/fp/zkb1xvsx6s1gy5dwtqm90krc0000gn/T/ipykernel_6049/4045680862.py:47: RuntimeWarning: invalid value encountered in scalar add
  y[u] += x[v]
/var/folders/fp/zkb1xvsx6s1gy5dwtqm90krc0000gn/T/ipykernel_6049/4045680862.py:68: RuntimeWarning: invalid value encountered in divide
  y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
/var/folders/fp/zkb1xvsx6s1gy5dwtqm90krc0000gn/T/ipykernel_6049/4045680862.py:78: RuntimeWarning: invalid value encountered in divide
  y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)


Original number of edges: 5278
New number of edges: 5805
Number of added edges: 527
Original Graph - Aggregate Commute Time: 1.5311
FOSR-modified Graph - Aggregate Commute Time: 1.2379
Percentage change in Aggregate Commute Time: -19.15%


In [ ]:
def process_dataset(dataset_name, num_graphs=1000):
    if dataset_name.lower() == 'zinc':
        dataset = ZINC(root='/tmp/ZINC', subset=True)
    elif dataset_name.lower() == 'qm9':
        dataset = QM9(root='/tmp/QM9')
    else:
        raise ValueError("Invalid dataset name. Choose 'ZINC' or 'QM9'.")
    
    results = []
    
    for i in range(min(num_graphs, len(dataset))):
        data = dataset[i]
        
        # Convert to NetworkX graph
        G_original = to_networkx(data, to_undirected=True)
        
        # Get edge index from the graph
        edge_index = np.array(list(G_original.edges())).T
        
        # Apply FOSR to get new edge index
        new_edge_index, new_edge_type = apply_fosr(edge_index)
        
        # Create a new graph with the rewired edges
        G_fosr = nx.Graph()
        G_fosr.add_nodes_from(range(data.num_nodes))
        new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
        G_fosr.add_edges_from(new_edges)
        
        # Calculate aggregate commute times for both graphs
        agg_commute_time_original = aggregate_commute_times(G_original)
        agg_commute_time_fosr = aggregate_commute_times(G_fosr)
        
        # Calculate the percentage change in aggregate commute time
        percent_change = ((agg_commute_time_fosr - agg_commute_time_original) / agg_commute_time_original) * 100
        
        results.append({
            'graph_index': i,
            'original_edges': G_original.number_of_edges(),
            'fosr_edges': G_fosr.number_of_edges(),
            'added_edges': G_fosr.number_of_edges() - G_original.number_of_edges(),
            'original_commute_time': agg_commute_time_original,
            'fosr_commute_time': agg_commute_time_fosr,
            'percent_change': percent_change
        })
        
        if i % 100 == 0:
            print(f"Processed {i+1} graphs...")
    
    return results

# Process ZINC dataset
print("Processing ZINC dataset...")
zinc_results = process_dataset('ZINC')

# Process QM9 dataset
print("\nProcessing QM9 dataset...")
qm9_results = process_dataset('QM9')

# Function to print summary statistics
def print_summary(results, dataset_name):
    avg_original_edges = np.mean([r['original_edges'] for r in results])
    avg_fosr_edges = np.mean([r['fosr_edges'] for r in results])
    avg_added_edges = np.mean([r['added_edges'] for r in results])
    avg_original_commute_time = np.mean([r['original_commute_time'] for r in results])
    avg_fosr_commute_time = np.mean([r['fosr_commute_time'] for r in results])
    avg_percent_change = np.mean([r['percent_change'] for r in results])
    
    print(f"\nSummary for {dataset_name} dataset:")
    print(f"Average original edges: {avg_original_edges:.2f}")
    print(f"Average FOSR edges: {avg_fosr_edges:.2f}")
    print(f"Average added edges: {avg_added_edges:.2f}")
    print(f"Average original commute time: {avg_original_commute_time:.4f}")
    print(f"Average FOSR commute time: {avg_fosr_commute_time:.4f}")
    print(f"Average percentage change in commute time: {avg_percent_change:.2f}%")

# Print summaries
print_summary(zinc_results, "ZINC")
print_summary(qm9_results, "QM9")

In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx, to_scipy_sparse_matrix
import networkx as nx
import numpy as np
from scipy.linalg import pinv
from math import inf
from numba import jit, int64
from torch_geometric.datasets import ZINC, QM9

def choose_edge_to_add(x, edge_index, degrees):
    # chooses edge (u, v) to add which minimizes y[u]*y[v]
    n = x.size
    m = edge_index.shape[1]
    y = x / ((degrees + 1) ** 0.5)
    products = np.outer(y, y)
    for i in range(m):
        u = edge_index[0, i]
        v = edge_index[1, i]
        products[u, v] = inf
    for i in range(n):
        products[i, i] = inf
    smallest_product = np.argmin(products)
    return (smallest_product % n, smallest_product // n)

def compute_degrees(edge_index, num_nodes=None):
    # returns array of degrees of all nodes
    if num_nodes is None:
        num_nodes = np.max(edge_index) + 1
    degrees = np.zeros(num_nodes)
    m = edge_index.shape[1]
    for i in range(m):
        degrees[edge_index[0, i]] += 1
    return degrees

def add_edge(edge_index, u, v):
    new_edge = np.array([[u, v],[v, u]])
    return np.concatenate((edge_index, new_edge), axis=1)

def adj_matrix_multiply(edge_index, x):
    # given an edge_index, computes Ax, where A is the corresponding adjacency matrix
    n = x.size
    y = np.zeros(n)
    m = edge_index.shape[1]
    for i in range(m):
        u = edge_index[0, i]
        v = edge_index[1, i]
        y[u] += x[v]
    return y

def compute_spectral_gap(edge_index, x):
	m = edge_index.shape[1]
	n = np.max(edge_index) + 1
	degrees = compute_degrees(edge_index, num_nodes=n)
	y = adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
	for i in range(n):
		if x[i] > 1e-9:
			return 1 - y[i]/x[i]
	return 0.

def _edge_rewire(edge_index, edge_type, x=None, num_iterations=50, initial_power_iters=50):
	m = edge_index.shape[1]
	n = np.max(edge_index) + 1
	if x is None:
		x = 2 * np.random.random(n) - 1
	degrees = compute_degrees(edge_index, num_nodes=n)
	for i in range(initial_power_iters):
		x = x - x.dot(degrees ** 0.5) * (degrees ** 0.5)/sum(degrees)
		y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
		x = y / np.linalg.norm(y)
	for I in range(num_iterations):
		i, j = choose_edge_to_add(x, edge_index, degrees=degrees)
		edge_index = add_edge(edge_index, i, j)
		degrees[i] += 1
		degrees[j] += 1
		edge_type = np.append(edge_type, 1)
		edge_type = np.append(edge_type, 1)
		x = x - x.dot(degrees ** 0.5) * (degrees ** 0.5)/sum(degrees)
		y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
		x = y / np.linalg.norm(y)
	return edge_index, edge_type, x

def edge_rewire(edge_index, x=None, edge_type=None, num_iterations=50, initial_power_iters=5):
    m = edge_index.shape[1]
    n = np.max(edge_index) + 1
    if x is None:
        x = 2 * np.random.random(n) - 1
    if edge_type is None:
        edge_type = np.zeros(m, dtype=np.int64)
    return _edge_rewire(edge_index, edge_type=edge_type, x=x, num_iterations=num_iterations, initial_power_iters=initial_power_iters)


# Apply the FOSR method
def apply_fosr(edge_index, rewire_fraction=0.1, min_iterations=10, max_iterations=1000):
    edge_type = np.zeros(edge_index.shape[1], dtype=np.int64)
    n = np.max(edge_index) + 1
    x = 2 * np.random.random(n) - 1
    
    # Calculate num_iterations based on the number of edges and rewire_fraction
    num_edges = edge_index.shape[1]
    num_iterations = int(num_edges * rewire_fraction)
    
    # Ensure num_iterations is within a reasonable range
    num_iterations = max(min_iterations, min(num_iterations, max_iterations))
    
    print(f"Number of iterations for FOSR: {num_iterations}")
    
    new_edge_index, new_edge_type, _ = edge_rewire(
        edge_index, 
        x=x, 
        edge_type=edge_type, 
        num_iterations=num_iterations, 
        initial_power_iters=5
    )
    
    return new_edge_index, new_edge_type

# Function to apply FOSR to any dataset
def apply_fosr_to_dataset(dataset, rewire_fraction=0.1, min_iterations=10, max_iterations=1000):
    data = dataset[0]
    G = to_networkx(data, to_undirected=True)
    edge_index = np.array(list(G.edges())).T
    
    new_edge_index, new_edge_type = apply_fosr(edge_index, rewire_fraction, min_iterations, max_iterations)
    
    new_G = nx.Graph()
    new_G.add_nodes_from(range(data.num_nodes))
    new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
    new_G.add_edges_from(new_edges)
    
    
    return new_edge_index, new_edge_type

def compute_commute_time(graph):
    """
    Compute the commute time for each pair of nodes in a graph.
    :param graph: A NetworkX graph object.
    :return: Commute time matrix (numpy array).
    """
    # Convert graph to adjacency matrix (scipy sparse format)
    adj_matrix = nx.adjacency_matrix(graph)
    
    # Compute degree matrix
    degrees = np.array(adj_matrix.sum(axis=1)).flatten()
    D = np.diag(degrees)
    
    # Compute Laplacian matrix L = D - A
    L = D - adj_matrix.toarray()
    
    # Compute the pseudoinverse of the Laplacian
    L_pseudo = pinv(L)
    
    # Compute commute time between all pairs of nodes
    num_nodes = L.shape[0]
    commute_time = np.zeros((num_nodes, num_nodes))
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            commute_time[i, j] = L_pseudo[i, i] + L_pseudo[j, j] - 2 * L_pseudo[i, j]
    
    return commute_time


def aggregate_commute_times(graph):
    """
    Aggregate the commute times for a single graph.
    :param graph: A NetworkX graph object.
    :return: Average commute time across the graph.
    """
    commute_times = compute_commute_time(graph)
    
    # Get upper triangular part of the commute time matrix (since it's symmetric)
    upper_triangle_indices = np.triu_indices_from(commute_times, k=1)
    commute_times_upper = commute_times[upper_triangle_indices]
    
    # Return average commute time
    return np.mean(commute_times_upper)

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Convert to NetworkX graph
G = to_networkx(data, to_undirected=True)

# Get edge index from the graph
edge_index = np.array(list(G.edges())).T

# Apply FOSR to get new edge index
new_edge_index, new_edge_type = apply_fosr_to_dataset(dataset)

# Create a new graph with the rewired edges
new_G = nx.Graph()
new_G.add_nodes_from(range(data.num_nodes))
new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
new_G.add_edges_from(new_edges)

# Print some statistics
print(f"Original number of edges: {G.number_of_edges()}")
print(f"New number of edges: {new_G.number_of_edges()}")
print(f"Number of added edges: {new_G.number_of_edges() - G.number_of_edges()}")

# Calculate aggregate commute times for both graphs
G_original = G  # Rename G to G_original for clarity
G_fosr = new_G  # Rename new_G to G_fosr for consistency

agg_commute_time_original = aggregate_commute_times(G_original)
agg_commute_time_fosr = aggregate_commute_times(G_fosr)

print(f"Original Graph - Aggregate Commute Time: {agg_commute_time_original:.4f}")
print(f"FOSR-modified Graph - Aggregate Commute Time: {agg_commute_time_fosr:.4f}")

# Calculate the percentage change in aggregate commute time
percent_change = ((agg_commute_time_fosr - agg_commute_time_original) / agg_commute_time_original) * 100

print(f"Percentage change in Aggregate Commute Time: {percent_change:.2f}%")

def process_dataset(dataset_name, num_graphs=1000):
    if dataset_name.lower() == 'zinc':
        dataset = ZINC(root='/tmp/ZINC', subset=False)
    elif dataset_name.lower() == 'qm9':
        dataset = QM9(root='/tmp/QM9')
    else:
        raise ValueError("Invalid dataset name. Choose 'ZINC' or 'QM9'.")
    
    results = []
    
    for i in range(min(num_graphs, len(dataset))):
        data = dataset[i]
        
        # Convert to NetworkX graph
        G_original = to_networkx(data, to_undirected=True)
        
        # Get edge index from the graph
        edge_index = np.array(list(G_original.edges())).T
        
        # Apply FOSR to get new edge index
        new_edge_index, new_edge_type = apply_fosr(edge_index)
        
        # Create a new graph with the rewired edges
        G_fosr = nx.Graph()
        G_fosr.add_nodes_from(range(data.num_nodes))
        new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
        G_fosr.add_edges_from(new_edges)
        
        # Calculate aggregate commute times for both graphs
        agg_commute_time_original = aggregate_commute_times(G_original)
        agg_commute_time_fosr = aggregate_commute_times(G_fosr)
        
        # Calculate the percentage change in aggregate commute time
        percent_change = ((agg_commute_time_fosr - agg_commute_time_original) / agg_commute_time_original) * 100
        
        results.append({
            'graph_index': i,
            'original_edges': G_original.number_of_edges(),
            'fosr_edges': G_fosr.number_of_edges(),
            'added_edges': G_fosr.number_of_edges() - G_original.number_of_edges(),
            'original_commute_time': agg_commute_time_original,
            'fosr_commute_time': agg_commute_time_fosr,
            'percent_change': percent_change
        })
        
        if i % 100 == 0:
            print(f"Processed {i+1} graphs...")
    
    return results

# Process ZINC dataset
print("Processing ZINC dataset...")
zinc_results = process_dataset('ZINC')

# Process QM9 dataset
print("\nProcessing QM9 dataset...")
qm9_results = process_dataset('QM9')

# Function to print summary statistics
def print_summary(results, dataset_name):
    avg_original_edges = np.mean([r['original_edges'] for r in results])
    avg_fosr_edges = np.mean([r['fosr_edges'] for r in results])
    avg_added_edges = np.mean([r['added_edges'] for r in results])
    avg_original_commute_time = np.mean([r['original_commute_time'] for r in results])
    avg_fosr_commute_time = np.mean([r['fosr_commute_time'] for r in results])
    avg_percent_change = np.mean([r['percent_change'] for r in results])
    
    print(f"\nSummary for {dataset_name} dataset:")
    print(f"Average original edges: {avg_original_edges:.2f}")
    print(f"Average FOSR edges: {avg_fosr_edges:.2f}")
    print(f"Average added edges: {avg_added_edges:.2f}")
    print(f"Average original commute time: {avg_original_commute_time:.4f}")
    print(f"Average FOSR commute time: {avg_fosr_commute_time:.4f}")
    print(f"Average percentage change in commute time: {avg_percent_change:.2f}%")

# Print summaries
print_summary(zinc_results, "ZINC")
print_summary(qm9_results, "QM9")

In [6]:
# second try



import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx, to_scipy_sparse_matrix
import networkx as nx
import numpy as np
from scipy.linalg import pinv
from math import inf
from numba import jit, int64
from torch_geometric.datasets import ZINC, QM9

def choose_edge_to_add(x, edge_index, degrees):
    # chooses edge (u, v) to add which minimizes y[u]*y[v]
    n = x.size
    m = edge_index.shape[1]
    y = x / ((degrees + 1) ** 0.5)
    products = np.outer(y, y)
    for i in range(m):
        u = edge_index[0, i]
        v = edge_index[1, i]
        products[u, v] = inf
    for i in range(n):
        products[i, i] = inf
    smallest_product = np.argmin(products)
    return (smallest_product % n, smallest_product // n)

def compute_degrees(edge_index, num_nodes=None):
    # returns array of degrees of all nodes
    if num_nodes is None:
        num_nodes = np.max(edge_index) + 1
    degrees = np.zeros(num_nodes)
    m = edge_index.shape[1]
    for i in range(m):
        degrees[edge_index[0, i]] += 1
    return degrees

def add_edge(edge_index, u, v):
    new_edge = np.array([[u, v],[v, u]])
    return np.concatenate((edge_index, new_edge), axis=1)

def adj_matrix_multiply(edge_index, x):
    # given an edge_index, computes Ax, where A is the corresponding adjacency matrix
    n = x.size
    y = np.zeros(n)
    m = edge_index.shape[1]
    for i in range(m):
        u = edge_index[0, i]
        v = edge_index[1, i]
        y[u] += x[v]
    return y

def compute_spectral_gap(edge_index, x):
	m = edge_index.shape[1]
	n = np.max(edge_index) + 1
	degrees = compute_degrees(edge_index, num_nodes=n)
	y = adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
	for i in range(n):
		if x[i] > 1e-9:
			return 1 - y[i]/x[i]
	return 0.

def _edge_rewire(edge_index, edge_type, x=None, num_iterations=50, initial_power_iters=50):
	m = edge_index.shape[1]
	n = np.max(edge_index) + 1
	if x is None:
		x = 2 * np.random.random(n) - 1
	degrees = compute_degrees(edge_index, num_nodes=n)
	for i in range(initial_power_iters):
		x = x - x.dot(degrees ** 0.5) * (degrees ** 0.5)/sum(degrees)
		y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
		x = y / np.linalg.norm(y)
	for I in range(num_iterations):
		i, j = choose_edge_to_add(x, edge_index, degrees=degrees)
		edge_index = add_edge(edge_index, i, j)
		degrees[i] += 1
		degrees[j] += 1
		edge_type = np.append(edge_type, 1)
		edge_type = np.append(edge_type, 1)
		x = x - x.dot(degrees ** 0.5) * (degrees ** 0.5)/sum(degrees)
		y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
		x = y / np.linalg.norm(y)
	return edge_index, edge_type, x

def edge_rewire(edge_index, x=None, edge_type=None, num_iterations=50, initial_power_iters=5):
    m = edge_index.shape[1]
    n = np.max(edge_index) + 1
    if x is None:
        x = 2 * np.random.random(n) - 1
    if edge_type is None:
        edge_type = np.zeros(m, dtype=np.int64)
    return _edge_rewire(edge_index, edge_type=edge_type, x=x, num_iterations=num_iterations, initial_power_iters=initial_power_iters)


# Apply the FOSR method
def apply_fosr(edge_index, rewire_fraction=0.1, min_iterations=10, max_iterations=1000):
    edge_type = np.zeros(edge_index.shape[1], dtype=np.int64)
    n = np.max(edge_index) + 1
    x = 2 * np.random.random(n) - 1
    
    # Calculate num_iterations based on the number of edges and rewire_fraction
    num_edges = edge_index.shape[1]
    num_iterations = int(num_edges * rewire_fraction)
    
    # Ensure num_iterations is within a reasonable range
    num_iterations = max(min_iterations, min(num_iterations, max_iterations))
    
    print(f"Number of iterations for FOSR: {num_iterations}")
    
    new_edge_index, new_edge_type, _ = edge_rewire(
        edge_index, 
        x=x, 
        edge_type=edge_type, 
        num_iterations=num_iterations, 
        initial_power_iters=5
    )
    
    return new_edge_index, new_edge_type

# Function to apply FOSR to any dataset
def apply_fosr_to_dataset(dataset, rewire_fraction=0.1, min_iterations=10, max_iterations=1000):
    data = dataset[0]
    G = to_networkx(data, to_undirected=True)
    edge_index = np.array(list(G.edges())).T
    
    new_edge_index, new_edge_type = apply_fosr(edge_index, rewire_fraction, min_iterations, max_iterations)
    
    new_G = nx.Graph()
    new_G.add_nodes_from(range(data.num_nodes))
    new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
    new_G.add_edges_from(new_edges)
    
    
    return new_edge_index, new_edge_type

def compute_commute_time(graph):
    """
    Compute the commute time for each pair of nodes in a graph.
    :param graph: A NetworkX graph object.
    :return: Commute time matrix (numpy array).
    """
    # Convert graph to adjacency matrix (scipy sparse format)
    adj_matrix = nx.adjacency_matrix(graph)
    
    # Compute degree matrix
    degrees = np.array(adj_matrix.sum(axis=1)).flatten()
    D = np.diag(degrees)
    
    # Compute Laplacian matrix L = D - A
    L = D - adj_matrix.toarray()
    
    # Compute the pseudoinverse of the Laplacian
    L_pseudo = pinv(L)
    
    # Compute commute time between all pairs of nodes
    num_nodes = L.shape[0]
    commute_time = np.zeros((num_nodes, num_nodes))
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            commute_time[i, j] = L_pseudo[i, i] + L_pseudo[j, j] - 2 * L_pseudo[i, j]
    
    return commute_time


def aggregate_commute_times(graph):
    """
    Aggregate the commute times for a single graph.
    :param graph: A NetworkX graph object.
    :return: Average commute time across the graph.
    """
    commute_times = compute_commute_time(graph)
    
    # Get upper triangular part of the commute time matrix (since it's symmetric)
    upper_triangle_indices = np.triu_indices_from(commute_times, k=1)
    commute_times_upper = commute_times[upper_triangle_indices]
    
    # Return average commute time
    return np.mean(commute_times_upper)

def apply_fosr(edge_index, rewire_fraction=0.1, min_iterations=10, max_iterations=1000):
    edge_type = np.zeros(edge_index.shape[1], dtype=np.int64)
    n = np.max(edge_index) + 1
    x = 2 * np.random.random(n) - 1
    
    num_edges = edge_index.shape[1]
    num_iterations = int(num_edges * rewire_fraction)
    num_iterations = max(min_iterations, min(num_iterations, max_iterations))
    
    new_edge_index, new_edge_type, _ = edge_rewire(
        edge_index, 
        x=x, 
        edge_type=edge_type, 
        num_iterations=num_iterations, 
        initial_power_iters=5
    )
    
    return new_edge_index, new_edge_type

def process_dataset(dataset_name):
    if dataset_name.lower() == 'zinc':
        dataset = ZINC(root='/tmp/ZINC', subset=False)  # Using the full dataset
    elif dataset_name.lower() == 'qm9':
        dataset = QM9(root='/tmp/QM9')
    else:
        raise ValueError("Invalid dataset name. Choose 'ZINC' or 'QM9'.")
    
    total_original_edges = 0
    total_fosr_edges = 0
    total_graphs = len(dataset)
    
    for i, data in enumerate(dataset):
        # Convert to NetworkX graph
        G_original = to_networkx(data, to_undirected=True)
        
        # Get edge index from the graph
        edge_index = np.array(list(G_original.edges())).T
        
        # Apply FOSR to get new edge index
        new_edge_index, _ = apply_fosr(edge_index)
        
        # Create a new graph with the rewired edges
        G_fosr = nx.Graph()
        G_fosr.add_nodes_from(range(data.num_nodes))
        new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
        G_fosr.add_edges_from(new_edges)
        
        total_original_edges += G_original.number_of_edges()
        total_fosr_edges += G_fosr.number_of_edges()
        
        if (i + 1) % 1000 == 0:
            print(f"Processed {i+1}/{total_graphs} graphs...")
    
    return total_original_edges, total_fosr_edges, total_graphs

# Function to print summary statistics
def print_summary(original_edges, fosr_edges, total_graphs, dataset_name):
    print(f"\nSummary for {dataset_name} dataset:")
    print(f"Total graphs processed: {total_graphs}")
    print(f"Number of original edges: {original_edges}")
    print(f"Number of edges in FOSR dataset: {fosr_edges}")
    print(f"Difference in edges: {fosr_edges - original_edges}")

# Process ZINC dataset
print("Processing ZINC dataset...")
zinc_original, zinc_fosr, zinc_graphs = process_dataset('ZINC')

# Process QM9 dataset
print("\nProcessing QM9 dataset...")
qm9_original, qm9_fosr, qm9_graphs = process_dataset('QM9')

# Print summaries
print_summary(zinc_original, zinc_fosr, zinc_graphs, "ZINC")
print_summary(qm9_original, qm9_fosr, qm9_graphs, "QM9")

Processing ZINC dataset...
/content/spectral.py:69: RuntimeWarning: divide by zero encountered in divide
  y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
/content/spectral.py:69: RuntimeWarning: invalid value encountered in divide
  y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
/content/spectral.py:79: RuntimeWarning: invalid value encountered in divide
  y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
/content/spectral.py:48: RuntimeWarning: invalid value encountered in scalar add
  y[u] += x[v]
Processed 10000/220011 graphs...
Processed 20000/220011 graphs...
Processed 30000/220011 graphs...
Processed 40000/220011 graphs...
Processed 50000/220011 graphs...
Processed 60000/220011 graphs...
Processed 70000/220011 graphs...
Processed 80000/220011 graphs...
Processed 90000/220011 graphs...
Processed 100000/220011 graphs...
Processed 110000/220011 graphs...
Processed 120000/220011 graphs...
Processed 130000/220011 graphs...
Processed 140000/220011 graphs...
Processed 150000/220011 graphs...
Processed 160000/220011 graphs...
Processed 170000/220011 graphs...
Processed 180000/220011 graphs...
Processed 190000/220011 graphs...
Processed 200000/220011 graphs...
Processed 210000/220011 graphs...
Processed 220000/220011 graphs...

Processing QM9 dataset...
Processed 10000/130831 graphs...
Processed 20000/130831 graphs...
Processed 30000/130831 graphs...
Processed 40000/130831 graphs...
Processed 50000/130831 graphs...
Processed 60000/130831 graphs...
Processed 70000/130831 graphs...
Processed 80000/130831 graphs...
Processed 90000/130831 graphs...
Processed 100000/130831 graphs...
Processed 110000/130831 graphs...
Processed 120000/130831 graphs...
Processed 130000/130831 graphs...

Summary for ZINC dataset:
Total graphs processed: 220011
Number of original edges: 5479717
Number of edges in FOSR dataset: 7678922
Difference in edges: 2199205
Average original commute time: 4.2192
Average FOSR commute time: 2.2814
Percentage change in commute time: -45.93%

Summary for QM9 dataset:
Total graphs processed: 130831
Number of original edges: 2441758
Number of edges in FOSR dataset: 3742733
Difference in edges: 1300975
Average original commute time: 2.7984
Average FOSR commute time: 1.2852
Percentage change in commute time: -54.07%

In [ ]:
#third try

import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx, to_scipy_sparse_matrix
import networkx as nx
import numpy as np
from scipy.linalg import pinv
from math import inf
from numba import jit, int64
from torch_geometric.datasets import ZINC, QM9

def choose_edge_to_add(x, edge_index, degrees):
    # chooses edge (u, v) to add which minimizes y[u]*y[v]
    n = x.size
    m = edge_index.shape[1]
    y = x / ((degrees + 1) ** 0.5)
    products = np.outer(y, y)
    for i in range(m):
        u = edge_index[0, i]
        v = edge_index[1, i]
        products[u, v] = inf
    for i in range(n):
        products[i, i] = inf
    smallest_product = np.argmin(products)
    return (smallest_product % n, smallest_product // n)

def compute_degrees(edge_index, num_nodes=None):
    # returns array of degrees of all nodes
    if num_nodes is None:
        num_nodes = np.max(edge_index) + 1
    degrees = np.zeros(num_nodes)
    m = edge_index.shape[1]
    for i in range(m):
        degrees[edge_index[0, i]] += 1
    return degrees

def add_edge(edge_index, u, v):
    new_edge = np.array([[u, v],[v, u]])
    return np.concatenate((edge_index, new_edge), axis=1)

def adj_matrix_multiply(edge_index, x):
    # given an edge_index, computes Ax, where A is the corresponding adjacency matrix
    n = x.size
    y = np.zeros(n)
    m = edge_index.shape[1]
    for i in range(m):
        u = edge_index[0, i]
        v = edge_index[1, i]
        y[u] += x[v]
    return y

def compute_spectral_gap(edge_index, x):
	m = edge_index.shape[1]
	n = np.max(edge_index) + 1
	degrees = compute_degrees(edge_index, num_nodes=n)
	y = adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
	for i in range(n):
		if x[i] > 1e-9:
			return 1 - y[i]/x[i]
	return 0.

def _edge_rewire(edge_index, edge_type, x=None, num_iterations=50, initial_power_iters=50):
	m = edge_index.shape[1]
	n = np.max(edge_index) + 1
	if x is None:
		x = 2 * np.random.random(n) - 1
	degrees = compute_degrees(edge_index, num_nodes=n)
	for i in range(initial_power_iters):
		x = x - x.dot(degrees ** 0.5) * (degrees ** 0.5)/sum(degrees)
		y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
		x = y / np.linalg.norm(y)
	for I in range(num_iterations):
		i, j = choose_edge_to_add(x, edge_index, degrees=degrees)
		edge_index = add_edge(edge_index, i, j)
		degrees[i] += 1
		degrees[j] += 1
		edge_type = np.append(edge_type, 1)
		edge_type = np.append(edge_type, 1)
		x = x - x.dot(degrees ** 0.5) * (degrees ** 0.5)/sum(degrees)
		y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
		x = y / np.linalg.norm(y)
	return edge_index, edge_type, x

def edge_rewire(edge_index, x=None, edge_type=None, num_iterations=50, initial_power_iters=5):
    m = edge_index.shape[1]
    n = np.max(edge_index) + 1
    if x is None:
        x = 2 * np.random.random(n) - 1
    if edge_type is None:
        edge_type = np.zeros(m, dtype=np.int64)
    return _edge_rewire(edge_index, edge_type=edge_type, x=x, num_iterations=num_iterations, initial_power_iters=initial_power_iters)


# Apply the FOSR method
def apply_fosr(edge_index, rewire_fraction=0.1, min_iterations=10, max_iterations=1000):
    edge_type = np.zeros(edge_index.shape[1], dtype=np.int64)
    n = np.max(edge_index) + 1
    x = 2 * np.random.random(n) - 1
    
    # Calculate num_iterations based on the number of edges and rewire_fraction
    num_edges = edge_index.shape[1]
    num_iterations = int(num_edges * rewire_fraction)
    
    # Ensure num_iterations is within a reasonable range
    num_iterations = max(min_iterations, min(num_iterations, max_iterations))
    
    print(f"Number of iterations for FOSR: {num_iterations}")
    
    new_edge_index, new_edge_type, _ = edge_rewire(
        edge_index, 
        x=x, 
        edge_type=edge_type, 
        num_iterations=num_iterations, 
        initial_power_iters=5
    )
    
    return new_edge_index, new_edge_type

# Function to apply FOSR to any dataset
def apply_fosr_to_dataset(dataset, rewire_fraction=0.1, min_iterations=10, max_iterations=1000):
    data = dataset[0]
    G = to_networkx(data, to_undirected=True)
    edge_index = np.array(list(G.edges())).T
    
    new_edge_index, new_edge_type = apply_fosr(edge_index, rewire_fraction, min_iterations, max_iterations)
    
    new_G = nx.Graph()
    new_G.add_nodes_from(range(data.num_nodes))
    new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
    new_G.add_edges_from(new_edges)
    
    
    return new_edge_index, new_edge_type

def compute_commute_time(graph):
    """
    Compute the commute time for each pair of nodes in a graph.
    :param graph: A NetworkX graph object.
    :return: Commute time matrix (numpy array).
    """
    # Convert graph to adjacency matrix (scipy sparse format)
    adj_matrix = nx.adjacency_matrix(graph)
    
    # Compute degree matrix
    degrees = np.array(adj_matrix.sum(axis=1)).flatten()
    D = np.diag(degrees)
    
    # Compute Laplacian matrix L = D - A
    L = D - adj_matrix.toarray()
    
    # Compute the pseudoinverse of the Laplacian
    L_pseudo = pinv(L)
    
    # Compute commute time between all pairs of nodes
    num_nodes = L.shape[0]
    commute_time = np.zeros((num_nodes, num_nodes))
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            commute_time[i, j] = L_pseudo[i, i] + L_pseudo[j, j] - 2 * L_pseudo[i, j]
    
    return commute_time


def aggregate_commute_times(graph):
    """
    Aggregate the commute times for a single graph.
    :param graph: A NetworkX graph object.
    :return: Average commute time across the graph.
    """
    commute_times = compute_commute_time(graph)
    
    # Get upper triangular part of the commute time matrix (since it's symmetric)
    upper_triangle_indices = np.triu_indices_from(commute_times, k=1)
    commute_times_upper = commute_times[upper_triangle_indices]
    
    # Return average commute time
    return np.mean(commute_times_upper)

def apply_fosr(edge_index, max_new_edges):
    edge_type = np.zeros(edge_index.shape[1], dtype=np.int64)
    n = np.max(edge_index) + 1
    x = 2 * np.random.random(n) - 1
    
    new_edge_index, new_edge_type, _ = edge_rewire(
        edge_index, 
        x=x, 
        edge_type=edge_type, 
        num_iterations=max_new_edges, 
        initial_power_iters=5
    )
    
    return new_edge_index, new_edge_type

def process_dataset(dataset_name):
    if dataset_name.lower() == 'zinc':
        dataset = ZINC(root='/tmp/ZINC', subset=False)  # Using the full dataset
    elif dataset_name.lower() == 'qm9':
        dataset = QM9(root='/tmp/QM9')
    else:
        raise ValueError("Invalid dataset name. Choose 'ZINC' or 'QM9'.")
    
    total_original_edges = 0
    total_fosr_edges = 0
    total_original_commute_time = 0
    total_fosr_commute_time = 0
    total_graphs = len(dataset)
    
    for i, data in enumerate(dataset):
        # Convert to NetworkX graph
        G_original = to_networkx(data, to_undirected=True)
        
        # Get edge index from the graph
        edge_index = np.array(list(G_original.edges())).T
        
        # Calculate max new edges (10% of original edges)
        max_new_edges = int(0.1 * G_original.number_of_edges())
        
        # Apply FOSR to get new edge index
        new_edge_index, _ = apply_fosr(edge_index, max_new_edges)
        
        # Create a new graph with the rewired edges
        G_fosr = nx.Graph()
        G_fosr.add_nodes_from(range(data.num_nodes))
        new_edges = list(zip(new_edge_index[0], new_edge_index[1]))
        G_fosr.add_edges_from(new_edges)
        
        # Calculate commute times
        original_commute_time = aggregate_commute_times(G_original)
        fosr_commute_time = aggregate_commute_times(G_fosr)
        
        total_original_edges += G_original.number_of_edges()
        total_fosr_edges += G_fosr.number_of_edges()
        total_original_commute_time += original_commute_time
        total_fosr_commute_time += fosr_commute_time
        
        if (i + 1) % 1000 == 0:
            print(f"Processed {i+1}/{total_graphs} graphs...")
    
    return total_original_edges, total_fosr_edges, total_original_commute_time, total_fosr_commute_time, total_graphs

# Function to print summary statistics
def print_summary(original_edges, fosr_edges, original_commute_time, fosr_commute_time, total_graphs, dataset_name):
    print(f"\nSummary for {dataset_name} dataset:")
    print(f"Total graphs processed: {total_graphs}")
    print(f"Number of original edges: {original_edges}")
    print(f"Number of edges in FOSR dataset: {fosr_edges}")
    print(f"Difference in edges: {fosr_edges - original_edges}")
    print(f"Percentage of edges added: {((fosr_edges - original_edges) / original_edges) * 100:.2f}%")
    print(f"Average original commute time: {original_commute_time / total_graphs:.4f}")
    print(f"Average FOSR commute time: {fosr_commute_time / total_graphs:.4f}")
    print(f"Percentage change in commute time: {((fosr_commute_time - original_commute_time) / original_commute_time) * 100:.2f}%")

# Process ZINC dataset
print("Processing ZINC dataset...")
zinc_original, zinc_fosr, zinc_original_ct, zinc_fosr_ct, zinc_graphs = process_dataset('ZINC')

# Process QM9 dataset
print("\nProcessing QM9 dataset...")
qm9_original, qm9_fosr, qm9_original_ct, qm9_fosr_ct, qm9_graphs = process_dataset('QM9')

# Print summaries
print_summary(zinc_original, zinc_fosr, zinc_original_ct, zinc_fosr_ct, zinc_graphs, "ZINC")
print_summary(qm9_original, qm9_fosr, qm9_original_ct, qm9_fosr_ct, qm9_graphs, "QM9")

Processing ZINC dataset...
/content/spectral.py:71: RuntimeWarning: divide by zero encountered in divide
  y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
/content/spectral.py:50: RuntimeWarning: invalid value encountered in scalar add
  y[u] += x[v]
/content/spectral.py:71: RuntimeWarning: invalid value encountered in divide
  y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
/content/spectral.py:81: RuntimeWarning: invalid value encountered in divide
  y = x + adj_matrix_multiply(edge_index, x / (degrees ** 0.5)) / (degrees ** 0.5)
Processed 10000/220011 graphs...
Processed 20000/220011 graphs...
Processed 30000/220011 graphs...
Processed 40000/220011 graphs...
Processed 50000/220011 graphs...
Processed 60000/220011 graphs...
Processed 70000/220011 graphs...
Processed 80000/220011 graphs...
Processed 90000/220011 graphs...
Processed 100000/220011 graphs...
Processed 110000/220011 graphs...
Processed 120000/220011 graphs...
Processed 130000/220011 graphs...
Processed 140000/220011 graphs...
Processed 150000/220011 graphs...
Processed 160000/220011 graphs...
Processed 170000/220011 graphs...
Processed 180000/220011 graphs...
Processed 190000/220011 graphs...
Processed 200000/220011 graphs...
Processed 210000/220011 graphs...
Processed 220000/220011 graphs...

Processing QM9 dataset...
Processed 10000/130831 graphs...
Processed 20000/130831 graphs...
Processed 30000/130831 graphs...
Processed 40000/130831 graphs...
Processed 50000/130831 graphs...
Processed 60000/130831 graphs...
Processed 70000/130831 graphs...
Processed 80000/130831 graphs...
Processed 90000/130831 graphs...
Processed 100000/130831 graphs...
Processed 110000/130831 graphs...
Processed 120000/130831 graphs...
Processed 130000/130831 graphs...

Summary for ZINC dataset:
Total graphs processed: 220011
Number of original edges: 5479717
Number of edges in FOSR dataset: 5926297
Difference in edges: 446580
Percentage of edges added: 8.15%
Average original commute time: 4.2192
Average FOSR commute time: 3.9500
Percentage change in commute time: -6.38%

Summary for QM9 dataset:
Total graphs processed: 130831
Number of original edges: 2441758
Number of edges in FOSR dataset: 2626425
Difference in edges: 184667
Percentage of edges added: 7.56%
Average original commute time: 2.7984
Average FOSR commute time: 2.4197
Percentage change in commute time: -13.53%

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

def train_and_evaluate(model, data, optimizer, num_epochs=200):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
    
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

def compare_performance(original_graph, fosr_graph):
    # Assume node features and labels are available in the graphs
    original_data = Data(x=original_graph.x, edge_index=original_graph.edge_index, y=original_graph.y)
    fosr_data = Data(x=fosr_graph.x, edge_index=fosr_graph.edge_index, y=fosr_graph.y)
    
    num_features = original_data.num_node_features
    num_classes = original_data.y.max().item() + 1

    original_model = GCN(num_features, num_classes)
    fosr_model = GCN(num_features, num_classes)

    original_optimizer = torch.optim.Adam(original_model.parameters(), lr=0.01)
    fosr_optimizer = torch.optim.Adam(fosr_model.parameters(), lr=0.01)

    original_acc = train_and_evaluate(original_model, original_data, original_optimizer)
    fosr_acc = train_and_evaluate(fosr_model, fosr_data, fosr_optimizer)

    print(f"Original Graph Accuracy: {original_acc:.4f}")
    print(f"FOSR-modified Graph Accuracy: {fosr_acc:.4f}")

# You would call this function for each graph in your dataset
# compare_performance(original_graph, fosr_graph)